### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [3]:
# read the data into pandas dataframes
BY_T24_GD_402a = pd.read_csv("lca/cleaned/402_BY_T24_GD_lca.csv")

# delete any Metazoa hits because they're trypsin
BY_T24_GD_402b = BY_T24_GD_402a[BY_T24_GD_402a.kingdom != 'Metazoa']

# keep only entries to the phylum level
BY_T24_GD_402 = BY_T24_GD_402b[BY_T24_GD_402b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(BY_T24_GD_402))

BY_T24_GD_402.head(6)

# of phylum peptides =  15


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
3,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,MNGELVDR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,MHLMHLTK,Dorea,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,...,Dorea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,VADVLSPR,Alphaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,TTTVSLPR,Trichomonas vaginalis,Eukaryota,NaN,NaN,NaN,Parabasalia,NaN,NaN,NaN,...,Trichomonas,NaN,NaN,NaN,Trichomonas vaginalis,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [5]:
BY_T24_GD_402_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/402A_BY_T24_GD_DN50_ILnaafs.csv")
BY_T24_GD_402_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/402B_BY_T24_GD_DN50_ILnaafs.csv")
#BY_T24_GD_402_NAAFc = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/402C_BY_T24_GD_DN50_ILnaafs.csv")


frames = [BY_T24_GD_402_NAAFa, BY_T24_GD_402_NAAFb]

BY_T24_GD_402_NAAF = pd.concat(frames, sort=False)

BY_T24_GD_402_NAAF.set_index('stripped_peptide')
BY_T24_GD_402_NAAF = BY_T24_GD_402_NAAF.loc[:, ~BY_T24_GD_402_NAAF.columns.str.contains('^Unnamed')]

BY_T24_GD_402_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(BY_T24_GD_402_NAAF))

print('column names:', BY_T24_GD_402_NAAF.columns)

BY_T24_GD_402_NAAF.head()

# of total peptides =  241
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,SPATLNSR,125000.0,15625.0
1,LSSPATLNSR,25200000.0,2520000.0
2,LSSPATLNSR,25200000.0,2520000.0
3,LSSPATLDSR,495000.0,49500.0
4,LSSPATLNSR,25200000.0,2520000.0


In [6]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(BY_T24_GD_402["peptide"], BY_T24_GD_402_NAAF["peptide"])

print(over)

['LATVLSPR' 'LATVNVPR' 'TTTVSLPR' 'VADVLSPR' 'VATVSPLR']


In [7]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

BY_T24_GD_402.set_index('peptide', inplace=True)
BY_T24_GD_402_NAAF.set_index('peptide', inplace=True)

BY_T24_GD_402_Phy = BY_T24_GD_402.join(BY_T24_GD_402_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(BY_T24_GD_402_Phy))

BY_T24_GD_402_Phy.head()

# of total phylum-level peptides =  30


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
AQQTADEANER,Proteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EELYMMK,Candidatus Saccharimonas,Bacteria,NaN,NaN,NaN,Candidatus Saccharibacteria,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9860000.0,1232500.0
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9860000.0,1232500.0
LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9860000.0,1232500.0


In [8]:
# write to a csv

BY_T24_GD_402_Phy.to_csv("lca/NAAF/BY_T24_GD_402_Phy_naaf.csv")